In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
df = pd.read_csv(r"C:\Users\anpar\Python\Pandas\Classification_train.csv")

In [3]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [4]:
class Standarize:
    
    mean = std = None
    
    def fit(self, X):
        self.mean = X.mean(axis=0) + 1e-30
        self.std = X.std(axis=0) + 1e-30
        
        
    def fit_transform(self, X):
        self.fit(X)
        X_scaled = (X - self.mean)/self.std
        return X_scaled
        

    def transform(self, X):
        try:
            X_scaled = (X - self.mean)/self.std
            return X_scaled
        except TypeError:
            raise TypeError('No data has been provided to calculate mean and standard deviation')



In [5]:
def grad_desc(X, y, learning_rate=0.01, n_iterations=100):
    m, n = X.shape
    w = np.zeros(n)
    
    for i in range(1, n_iterations+1):
        
        z = X.dot(w)
        y_pred = sigmoid(z)
        w += learning_rate*(y - y_pred).T.dot(X)/m
        
        if i%10 == 0:
            loss_func = -np.mean(y * np.log(y_pred + 1e-8) + (1 - y) * np.log(1 - y_pred + 1e-8))/m
            print(f'{i}th iteration done. Loss = {loss_func}')

    return w

In [6]:
def step(z):
    return 1 if z >= 0 else 0

In [7]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [8]:
def one_hot_encoding(y):
    classes = np.unique(y)
    y_encoded = np.zeros(y.shape[0])
    
    for i in classes:
        y_df = pd.DataFrame(y).loc[y == i].copy()
        y_n = pd.DataFrame(np.zeros(y.shape[0]))
        y_n.loc[y_df.index] = 1
        y_encoded = np.c_[y_encoded, y_n]
    
    y_encoded = np.delete(y_encoded, 0, 1)
    
    return y_encoded

In [9]:
def calc_accuracy(y_test, y_preds):
    m, n = y_test.shape
    y_total = m*n
    
    y_bool = (y_test == y_preds)
    y_correct = np.count_nonzero(y_bool == True)
    
    accuracy = y_correct/y_total
    return accuracy
    

In [10]:
class LogisticRegression:
    def fit(self, X_train, y_train, learning_rate=0.01, n_iterations=100):
        self.w_s = np.zeros((X_train.shape[1], 1))
        for k in range(y_train.shape[1]):
            y_train_col = y_train[:, k]
            w = grad_desc(X_train, y_train_col, learning_rate=learning_rate, n_iterations=n_iterations)
            self.w_s = np.insert(self.w_s, -1, w, axis=1)
            print(w.shape)
        self.w_s = np.delete(self.w_s, -1, axis=1)

    def predict(self, X_test):
        step_brute = np.vectorize(step)
        self.y_pred = step_brute(X_test.dot(self.w_s))
        return self.y_pred
        

In [11]:
n_iterations = 100
learning_rate = 0.01

In [12]:
# scalar = Standarize()
# X = scalar.fit_transform(X)
# X

In [13]:
X = np.insert(X, 0, 1, axis=1)

In [14]:
y_en = one_hot_encoding(y)

In [15]:
X_train, y_train = np.array(X[:int(0.8*X.shape[0]), :]), np.array(y_en[:int(0.8*X.shape[0]), :])
X_test, y_test = np.array(X[int(0.8*X.shape[0]):, :]), np.array(y_en[int(0.8*X.shape[0]):, :])

In [16]:
y_test = np.array(y_en[int(0.8*X.shape[0]):, :])

In [17]:
regressor = LogisticRegression()
w_s = regressor.fit(X_train, y_train, n_iterations=100)

C:\Users\anpar\AppData\Local\Temp\ipykernel_34780\3692174223.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-z))


10th iteration done. Loss = 6.644538851324608e-05
20th iteration done. Loss = 1.5020458236913031e-05
30th iteration done. Loss = 1.2852618429357496e-05
40th iteration done. Loss = 1.1304249506245099e-05
50th iteration done. Loss = 9.94541772251843e-06
60th iteration done. Loss = 8.708848263713403e-06
70th iteration done. Loss = 7.86065231314471e-06
80th iteration done. Loss = 7.463389796119461e-06
90th iteration done. Loss = 7.306250630435708e-06
100th iteration done. Loss = 7.151933636748867e-06
(785,)
10th iteration done. Loss = 1.535388754712788e-05
20th iteration done. Loss = 9.438622399992688e-06
30th iteration done. Loss = 7.186748315913477e-06
40th iteration done. Loss = 5.742339000272253e-06
50th iteration done. Loss = 5.056572442347027e-06
60th iteration done. Loss = 4.799619333593884e-06
70th iteration done. Loss = 4.525237869081727e-06
80th iteration done. Loss = 4.102305720546987e-06
90th iteration done. Loss = 3.6971191473632825e-06
100th iteration done. Loss = 3.390269264

In [18]:
y_pred = regressor.predict(X_test)

In [19]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [20]:
y_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [21]:
calc_accuracy(y_test, y_pred)

0.9866

In [38]:
new_df = pd.read_csv(r"C:\Users\anpar\Python\ML_Bootcamp_Aadi\Algo\Test_Datasets\Classification_test.csv")

In [43]:
X_new = new_df.iloc[:, 1:]
X_new = np.insert(X_new, 0, 1, axis=1)

In [49]:
y_en = regressor.predict(X_new)

In [50]:
y_new = np.argmax(y_en, axis=1)

In [51]:
new_df = np.insert(new_df, 1, y_new, 1)

In [55]:
labeled_df = pd.DataFrame(new_df, columns=['ID']+['label']+['pixel'+str(i) for i in range(784)])

In [56]:
labeled_df.to_csv(r"C:\Users\anpar\Python\ML_Bootcamp_Aadi\Algo\Test_Datasets\Labeled_Logistic_Classification_test.csv", index=False)